# COGS 118B - Final Project

# Applying machine learning techniques to baked-goods recipe data

## Group members

- Zixian Cai
- Richard Masser-Frye
- Vinuthna Hasthi
- Pranav Nair
- Jesus Tello

# Abstract 
How does a recipe define a food item? What makes a cake a cake? In this project, we sought to cluster recipes based on their ingredients and other variables, and determine whether a machine learning model can distinguish recipes with regard to the type of food item that the recipe is supposed to describe. To narrow the scope of the problem, we focussed on baked goods, and attempting to cluster the data by type of baked good (cakes, cookies, muffins). The data we used is a set of recipes, with each recipe having a title and set of ingredients (with associated quantities). Data was extracted from the quantities of each ingredient, and was fed into a Gaussian mixture model (GMM) as well as other models. We attempted to measure success via the efficacy of the clustering algorithms via Bayesian information criterion (BIC).

Ultimately, separating the data into clusters proved more difficult than we expected, but we were still able to observe some interesting trends among the dataset. 

# Background

Broadly, recipes are sets of instructions for the purpose of preparing specific dishes. They generally include some information about the dish, the ingredients used in making the dish, and how to assemble or process those ingredients. In particular, baked goods are dishes (such as cookies, cakes, etc) that contain flour as a primary ingredient and involve combining ingredients to make a batter which is placed into an oven to bake. The consistency of the baked good generally depends on the proportions of flour, egg, sugar, butter, and baking powder used in the batter, and on how long and at what temperature it is baked.

Recipe datasets have previously been used in machine learning research. Often, the research is not particularly focused on recipes, but uses them for their advantageous qualities (most recipes share a similar structure, they can be easily scraped from websites, they can't be copyrighted, etc). For example, a 2017 paper by Yang et al.<a name="yang"></a>[<sup>[1]</sup>](#yangnote) used machine learning to analyze "reference expressions" (phrases that refer to previously mentioned objects in a text) and employed instructions from recipes as an example of this. Other published works are more explicitly food-focused; Herranz et al.<a name="herranz"></a>[<sup>[2]</sup>](#herranznote) provides an overview of techniques that can be applied to recipes, ingredient lists, and even images of food. The paper associated with the RecipeNLG dataset by Bien et al.<a name="recipenlg"></a>[<sup>[3]</sup>](#recipenlgnote) also provides an overview of previous efforts in machine learning on recipes, including the two aforementioned papers.

# Problem Statement

The problem at hand is the categorization of baked-goods recipes based on their ingredients. Though they are not explicitly numerical, these attributes have aspects that can be abstracted into numbers; specifically, the quantities of each ingredient in grams. Each recipe can thus be expressed as a vector, and clustering techniques (such as GMM or hierarchical clustering) can be attempted to the dataset to sort the recipes into groups. We can also use high-dimensional data visualization techniques like PCA or manifold learning to get a sense of the overall shape of the dataset. Finally, we can measure the success of the clustering by comparing the predicted labels to the true labels found in the original dataset. 

# Data

We used a dataset called RecipeNLG created by researchers at Poznan University of Technology.<a name="recipenlg2"></a>[<sup>[3]</sup>](#recipenlgnote)

- Links: [RecipeNLG official website](https://recipenlg.cs.put.poznan.pl/), [RecipeNLG Kaggle page](https://www.kaggle.com/datasets/paultimothymooney/recipenlg)
- RecipeNLG contains 2,231,142 recipes in all, but for our project we filtered it down to 235,762 recipes that had "cake", "cookie" or "muffin" in the title. After processing, it was further filtered down to 145,087 data points.
- Post-processing, an observation consists of quantities of each of the following ingredients, in grams:
    - egg
    - flour
    - sugar
    - butter
    - vanilla extract
    - milk
    - evaporated milk
    - condensed milk
    - shortening
    - powdered sugar
    - cornmeal
    - baking soda
    - baking powder
    - oats
- Whether a particular datapoint is from a cake, cookie, or muffin recipe is stored in a column called 'category', and is treated as a ground truth observation (i.e. it is not used for unsupervised tasks)

The initial filtering of the data for baked-goods recipes was done in [filter_recipes.ipynb](https://github.com/richmass1/group_template/blob/main/filter_recipes.ipynb), by selecting recipes that had in their titles "cake", "cookie", or "muffin". Converting the recipes into data points was done in [process_recipes.ipynb](https://github.com/richmass1/group_template/blob/main/process_recipes.ipynb), by searching each ingredient string (for example, `'1/2 c. sugar'`) for an ingredient name, then extracting the number and unit, converting the number to a float, then converting the unit to grams. 

After processing, the data was further filtered by removing data points that had no flour, data points with invalid values and datapoints with values that seemed overly large.

# Proposed Solution

Our strategy was to try a number of ML algorithms on the dataset, and see what did the best with our particular dataset. An early idea was to first reduce dimensionality, then use GMM to cluster the data points, and see whether GMM could accurately separate the recipe into their true categories. We also decided to try alternative clustering techniques like hierarchical clustering and spectral clustering. Finally, we tried PCA, UMAP, and t-SNE as data visualization techniques.

# Evaluation Metrics

For clustering techniques like GMM and hierarchical clustering, we initially proposed using Bayesian information criterion (BIC) to evaluate the clustering itself, and adjusted Rand index (ARI) to evaluate how well it assigned data points. However, it turned out that the data was not conducive to simple clustering, rendering ARI unnecessary. 

For dimensionality reduction algorithms (PCA, t-SNE, UMAP) we used a more supervised approach. For all 3 techniques, we used silhouette scores with true labels to gauge how well it placed recipes of the same type near each other. For PCA and UMAP, we trained a k-nearest neighbors (kNN) classifier on the embedded data and then used ARI to evaluate the performance of the classifier.

# Results

### Starting out, getting a picture of the data with PCA
We wanted to get an idea of the overall shape of the dataset, so to start off, we did a simple dimensionality reduction with PCA:

<img width=400 src="./images/PCA_projection.png">

(image from [this notebook](https://github.com/richmass1/group_template/blob/main/118b%20PCA%20project.ipynb))

It was immediately clear that the data did not form clusters that were obvious enough to show up on a PCA projection. However, this was also the case for the MNIST dataset, which in its 2-dimensional projection did not form separate clusters. To see whether PCA helped separate data according to its true labels, we also tried rendering it with categories as labels:

<img width=400 src="./images/PCAwithlabels.png">

(image from [this notebook](https://github.com/richmass1/group_template/blob/main/PCA_UMAP_tSNE.ipynb))

In this plot, the true categories are somewhat apparent, although they have a large amount of overlap. Reduction to two dimensions via PCA produced a visible amount of seaparation, but not enough to make clearly delineated clusters. Finally, we also tried a 3-dimensional PCA:

<img width=450 src="./images/PCA3d.png">

(image from [this notebook](https://github.com/richmass1/group_template/blob/main/118b%20PCA%20project.ipynb))

However, the 3D plot also showed significant overlap between the distributions of the true categories.

## Clustering Techniques

### Gaussian mixture model (GMM)
Next, we tried applying a GMM to the dimension-reduced dataset, using a grid search to find the optimal combination of:
- Covariance type (full, tied, spherical)
- Number of PCA dimensions
- Number of Gaussian clusters

with BIC as an evaluation metric.

<img width=850 src="./images/GMM.png">

(image from [this notebook](https://github.com/richmass1/group_template/blob/main/118B%20GMM%20Project.ipynb))

What we found was that even as we increased the number of principle components, the data wasn't falling into three distinct clusters. Instead of getting the sought-after elbow plot that levels out at the optimal number, the BIC continued decreasing even while increasing the complexity of the model (i.e., 6 Gaussian clusters and 6 principle components).

### Hierarchical clustering
We also tried hierarchical clustering on the dataset reduced to 2 dimensions. As can be seen in the plot, the clusters that it produces are somewhat arbitrary, and didn't look like they corresponded with the true categories.

<img width=400 src="./images/hierarchical.png">

(image from [this notebook](https://github.com/richmass1/group_template/blob/main/Hierarchical%20Spectral%20Clustering.ipynb))

## Understanding the dataset: dimensionality reduction
Since PCA and simple clustering techniques were largely unsuccessful in producing a satisfying segmentation of the dataset, we tried transforming it with the manifold learning algorithms t-SNE and UMAP. With the MNIST dataset, UMAP produced a 2 dimensional rendering that worked pretty well to put data points representing the same digit near each other; we hoped that on our dataset UMAP and t-SNE might give us a visualization that does better at putting recipes of the same category nearer to each other.

### t-SNE
<img width=400 src="./images/tSNE.png">

What we saw with the t-SNE plot was it grouped the data points into a bunch of separate clumps, but each clump seemed to be mostly composed of the same category of recipe.

### UMAP
<img width=400 src="./images/UMAP.png">

The same was also true of the UMAP visualization, in that it seemed to create a series of clumps, and that each clump was mostly composed of the same category of recipe, but like t-SNE, it didn't necessarily put cookie-clumps near cookie-clumps or cake-clumps near other cake-clumps.

To get a more numeric interpretation of the results of the manifold learning algorithms, we used mean silhouette scores to evaluate the two techniques above as well as a simple 2-D PCA plot. 

### Evaluating DR with silhouette scores
```
2d PCA silhouette score:
0.04278151389636587
UMAP silhouette score:
-0.12163689
t-SNE silhouette score:
0.0022150672
```
The mean silhouette scores for all 3 dimensionality reduction techniques are quite low. We also observed that the manifold learning techniques had significantly worse mean scores than PCA, and we theorize that this is because PCA, like silhouette scores, is a global operation, while manifold techniques focus on local structure and fail to preserve global structure. 

We then used a kNN classifier on the UMAP representation, as well as the original dataset and 2-D PCA for comparison:

### Adjusted Rand index on UMAP and 2-D PCA
```
2-D PCA + kNN, adjusted Rand index:
0.340675938219707
kNN w/o dim reduction, adjusted Rand index:
0.523566902614766
UMAP + kNN, adjusted Rand index:
0.48453393771033726
```
We observed that the UMAP-trained classifier performed better than the PCA-trained classifier, and theorize that this is because kNN pays more attention to local structure, which UMAP does better to preserve. It does not perform as well as the unreduced dataset, as information is inevitably lost when the dimensionality is reduced.

(images and data generated by [this notebook](https://github.com/richmass1/group_template/blob/main/PCA_UMAP_tSNE.ipynb))

# Discussion

### Interpreting the result
In our analysis, we noticed the data tended to form numerous, subtle clusters, something PCA couldn't quite grasp due to its focus on capturing the broader picture rather than the fine details. On the other hand, methods like UMAP and t-SNE, which hone in on local relationships, revealed these smaller groupings more effectively. This difference in the clustering results from each method highlights the nuanced nature of our dataset, suggesting that the more granular cluster formations might be lost under the shadow of larger data trends when using PCA. However, even the techniques UMAP and t-SNE didn't necessarily form distinct separate groups for each true label of cake, cookies, and muffins. This is probably due to the fact that the recipes themselves that create cake, cookies, and muffins are all relatively similar, and may even be extremely similar depending on the specific desired type of cookie/cake/muffin.

### Limitations
Processing the dataset proved to be more difficult than expected, due to the nature of recipes being created without a central standard for ingredient formatting. Despite trying to account for variability in the way ingredient lists were formatted, we were still unable to perfectly represent the recipes in our dataset. We also noted the unequal representation of muffins compared to cakes and cookies.

### Ethics & Privacy

For the most part, we believe we will not run into any ethics or privacy issues. We acknowledge potential ethical concerns with the cultural sensitivities associated with foods. When it comes to the ethical concerns of data privacy, we will ensure that we will not breach any privacy rules or laws when it comes to personal recipes. The data used in our analysis will strictly be obtained from publicly available sources, and in doing so, private recipes will not be breached. Overall, our team will address any emerging issue or concerns when it comes to ethics and privacy. We will adapt our project if we see any unethical behavior done by our team, algorithm, or data.

### Conclusion

The main point that we concluded is the individual categories like cookies and cakes were more diverse than we expected, and displayed a degree of inter-category similarity. In each category, there also seemed to be intra-category clusters that may not have been necessarily similar to other ingredients in that category (for example, consider how cheesecakes are alike to each other, but very different from other kinds of cakes).

# Footnotes
<a name="yangnote"></a>1.[^](#yang): Zichao Yang, Phil Blunsom, Chris Dyer, and Wang Ling. (9 Aug 2017) Reference-Aware Language Models. https://arxiv.org/pdf/1611.01628.pdf<br> 

<a name="herranznote"></a>2.[^](#herranz): Luis Herranz, Weiqing Min and Shuqiang Jiang. (22 Jan 2018) Food recognition and recipe analysis: integrating visual content, context and external knowledge. https://arxiv.org/pdf/1801.07239.pdf<br> 

<a name="recipenlgnote"></a>3.[^](#recipenlg), [^](#recipenlg2): Michał Bień, Michał Gilski, Martyna Maciejewska, Wojciech Taisner, Dawid Wisniewski, and Agnieszka Lawrynowicz. (Dec 2020) RecipeNLG: A Cooking Recipes Dataset for Semi-Structured Text Generation. https://aclanthology.org/2020.inlg-1.4/<br>